# 对数几率回归

In [1]:
import pandas as pd
import numpy as np

In [10]:
data = pd.read_csv('./tt/train.csv')

In [11]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [12]:
data = data[['Survived', 'Pclass','Sex', 'Age', 'SibSp','Parch', 'Fare', 'Embarked']]

In [13]:
data = data.fillna(0)

In [15]:
data['Sex'] = pd.factorize(data.Sex)[0]

In [18]:
data['p1'] = np.array(data['Pclass'] ==1).astype(np.float64)
data['p2'] = np.array(data['Pclass'] ==2).astype(np.float64)
data['p3'] = np.array(data['Pclass'] ==3).astype(np.float64)

In [20]:
del data['Pclass']

In [21]:
data.Embarked.unique()

array(['S', 'C', 'Q', 0], dtype=object)

In [22]:
data['e1'] = np.array(data['Embarked'] =='S').astype(np.float64)
data['e2'] = np.array(data['Embarked'] =='C').astype(np.float64)
data['e3'] = np.array(data['Embarked'] =='Q').astype(np.float64)

In [23]:
del data['Embarked']

In [24]:
data.columns

Index(['Survived', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'p1', 'p2', 'p3',
       'e1', 'e2', 'e3'],
      dtype='object')

In [28]:
import tensorflow as tf

In [29]:
data.Sex.values

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 1,

In [30]:
data_data = np.stack([data.Sex.values.astype(np.float64),data.Age.values.astype(np.float64),data.SibSp.values.astype(np.float64),
                      data.Parch.values.astype(np.float64),data.Fare.values.astype(np.float64),data.p1.values,
                      data.p2.values,data.p3.values,data.e1.values,data.e2.values,data.e3.values]).T

In [31]:
data_target = np.reshape(data.Survived.values.astype(np.float64),(891,1))

In [32]:
np.shape(data_target)

(891, 1)

In [33]:
np.shape(data_data)

(891, 11)

In [34]:
##定义网络

In [35]:
x = tf.placeholder("float", shape=[None,11])
y = tf.placeholder("float", shape=[None,1])

In [36]:
weight = tf.Variable(tf.random_normal([11,1]))
bias = tf.Variable(tf.random_normal([1]))
output = tf.matmul(x,weight) + bias
pred = tf.cast(tf.sigmoid(output)>0.5,tf.float32)

In [37]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y,logits = output))

In [38]:
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [39]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred,y), tf.float32))

In [40]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [41]:
for i in range(10000):
    for n in range(len(data_target)//100):
        index = np.random.permutation(len(data_target))
        data_data = data_data[index]
        data_target = data_target[index]
        batch_xs = data_data[n:n+100]
        batch_ys = data_target[n:n+100]
        sess.run(train_step,feed_dict={x: batch_xs, y: batch_ys})
    if i%1000 == 0:
        print(sess.run((loss,accuracy),feed_dict={x: batch_xs, y: batch_ys}))

(22.239229, 0.37)
(0.74265444, 0.54000002)
(0.51971304, 0.73000002)
(0.57914847, 0.69999999)
(0.45741248, 0.81999999)
(0.39966094, 0.87)
(0.45113933, 0.75999999)
(0.41452357, 0.83999997)
(0.38988754, 0.83999997)
(0.38947707, 0.82999998)


In [42]:
data_test = pd.read_csv('./tt/test.csv')
data_test = data_test.fillna(0)
data_test['Sex'] = pd.factorize(data_test.Sex)[0]
data_test['p1'] = np.array(data_test['Pclass'] ==1).astype(np.float64)
data_test['p2'] = np.array(data_test['Pclass'] ==2).astype(np.float64)
data_test['p3'] = np.array(data_test['Pclass'] ==3).astype(np.float64)
data_test['e1'] = np.array(data_test['Embarked'] =='S').astype(np.float64)
data_test['e2'] = np.array(data_test['Embarked'] =='C').astype(np.float64)
data_test['e3'] = np.array(data_test['Embarked'] =='Q').astype(np.float64)
test_data = np.stack([data_test.Sex.values.astype(np.float64),data_test.Age.values.astype(np.float64),data_test.SibSp.values.astype(np.float64),
                      data_test.Parch.values.astype(np.float64),data_test.Fare.values.astype(np.float64),data_test.p1.values,
                      data_test.p2.values,data_test.p3.values,data_test.e1.values,data_test.e2.values,data_test.e3.values]).T

In [43]:
test_lable = pd.read_csv('./tt/gender.csv')

In [44]:
test_lable = np.reshape(test_lable.Survived.values.astype(np.float64),(418,1))

In [45]:
sess.run(accuracy,feed_dict={x: test_data, y: test_lable})

0.93540668